# ComfyUI (GGUF) ? Wan2.2 14B Combo (I2V + First/Last)

This notebook includes both Wan2.2 modes in one place:
- Image-to-Video (I2V) 14B high/low noise GGUF
- First-Frame-to-Last-Frame (camera/control style) 14B high/low noise GGUF

Quick start:
1. Run all cells top-to-bottom.
2. Provide HF token when prompted.
3. In ComfyUI, load the corresponding workflow JSON for the mode you want.


## 1) Installation
Install ComfyUI, ComfyUI-Manager, ComfyUI-GGUF, and swap for better Colab stability.


In [ ]:
# @title 1) Install ComfyUI + Manager + ComfyUI-GGUF (with swap)
import os

# Small protection against RAM-related crashes
if not os.path.exists('/swapfile'):
    print('Creating swap (8GB)...')
    !sudo fallocate -l 8G /swapfile
    !sudo chmod 600 /swapfile
    !sudo mkswap /swapfile
    !sudo swapon /swapfile
    print('Swap enabled.')

# Useful packages
!apt-get -y update -qq
!apt-get -y install -qq aria2 ffmpeg

# More predictable CUDA allocator (sometimes helps reduce fragmentation)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# ComfyUI
if not os.path.exists('/content/ComfyUI'):
    print('Cloning ComfyUI...')
    !git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI

%cd /content/ComfyUI

print('Installing python requirements...')
!pip install -U pip
!pip install -r requirements.txt

# Attention acceleration pack (T4-safe)
ENABLE_ACCEL_PACK = True  # Set False to skip optional attention accelerators.

if ENABLE_ACCEL_PACK:
    import sys
    import subprocess

    def pip_try(spec):
        print(f"Installing optional accelerator: {spec}")
        return subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', spec], check=False).returncode == 0

    import torch
    gpu_name = 'CPU'
    cc_major, cc_minor = (0, 0)
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        cc_major, cc_minor = torch.cuda.get_device_capability(0)

    print(f"GPU detected: {gpu_name} (sm{cc_major}{cc_minor})")

    x_ok = pip_try('xformers>=0.0.28.post3')
    print('xformers:', 'OK' if x_ok else 'FAILED (continuing)')

    # FlashAttention-3 is Hopper-only; FlashAttention-2 and SageAttention target Ampere+.
    if cc_major >= 8:
        fa_ok = pip_try('flash-attn>=2.7.0.post2')
        sage_ok = pip_try('sageattention>=2.1.1')
        print('flash-attn:', 'OK' if fa_ok else 'FAILED (continuing)')
        print('sageattention:', 'OK' if sage_ok else 'FAILED (continuing)')
    else:
        print('Skipping flash-attn and sageattention on this GPU: T4/Turing (sm75) uses xformers path.')
else:
    print('Attention acceleration pack disabled by user (ENABLE_ACCEL_PACK=False).')


# ComfyUI-Manager
if not os.path.exists('custom_nodes/ComfyUI-Manager'):
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git custom_nodes/ComfyUI-Manager

# ComfyUI-GGUF
if not os.path.exists('custom_nodes/ComfyUI-GGUF'):
    print('Installing ComfyUI-GGUF...')
    !git clone https://github.com/city96/ComfyUI-GGUF.git custom_nodes/ComfyUI-GGUF
    !pip install -r custom_nodes/ComfyUI-GGUF/requirements.txt

# ComfyUI-KJNodes
if not os.path.exists('custom_nodes/comfyui-kjnodes'):
    print('Installing ComfyUI-KJNodes...')
    !git clone https://github.com/kijai/ComfyUI-KJNodes.git custom_nodes/comfyui-kjnodes
if os.path.exists('custom_nodes/comfyui-kjnodes/requirements.txt'):
    !pip install -r custom_nodes/comfyui-kjnodes/requirements.txt



# Hugging Face authentication (interactive prompt)
!pip install -q huggingface_hub
from getpass import getpass
from huggingface_hub import login

hf_token = getpass('Enter Hugging Face token: ').strip()
if not hf_token:
    raise RuntimeError('Hugging Face token is required for this notebook run.')

login(token=hf_token, add_to_git_credential=False)
os.environ['HUGGINGFACE_TOKEN'] = hf_token
os.environ['HF_TOKEN'] = hf_token
print('HF auth: OK')

# Civitai authentication (for LoRA downloads)
civitai_token = getpass('Enter Civitai API token (optional, press Enter to skip): ').strip()
if civitai_token:
    os.environ['CIVITAI_API_TOKEN'] = civitai_token
    print('Civitai auth: OK')
else:
    print('Civitai token not set. Civitai downloads may fail (403).')

print('Done.')


## 2) Quant Settings
Choose GGUF quant defaults and review rough file-size/VRAM estimates.


In [ ]:
# @title 2) Settings: GGUF quant selection + size/VRAM table
import math

# ---- Auto quant by VRAM ----
AUTO_QUANT_BY_VRAM = True
AUTO_QUANT_VRAM_FRACTION = 0.9
AUTO_VRAM_FALLBACK_GB = 14.0


# ---- Mode toggles ----
DOWNLOAD_I2V = True
DOWNLOAD_FLF2V = False

# Wan2.2 I2V 14B GGUF (both are required)
I2V_HIGH_QUANT = 'Q2_K'  # options: Q2_K, Q3_K_S, Q3_K_M, Q3_K_L, Q4_K_S, Q4_K_M, Q5_K_S, Q5_K_M, Q6_K, Q8_0
I2V_LOW_QUANT  = 'Q2_K'  # options: Q2_K, Q3_K_S, Q3_K_M, Q3_K_L, Q4_K_S, Q4_K_M, Q5_K_S, Q5_K_M, Q6_K, Q8_0

# Wan2.2 First/Last (Fun Camera A14B GGUF, both are required)
FLF2V_HIGH_QUANT = 'Q2_K'  # options: Q2_K, Q3_K_S, Q3_K_M, Q4_0, Q4_1, Q4_K_S, Q4_K_M, Q5_0, Q5_1, Q5_K_S, Q5_K_M, Q6_K, Q8_0
FLF2V_LOW_QUANT  = 'Q2_K'  # options: Q2_K, Q3_K_S, Q3_K_M, Q4_0, Q4_1, Q4_K_S, Q4_K_M, Q5_0, Q5_1, Q5_K_S, Q5_K_M, Q6_K, Q8_0

# Text encoder GGUF for Wan family
UMT5_QUANT = 'Q3_K_S'  # options: Q3_K_S, Q3_K_M, Q4_K_S, Q4_K_M, Q5_K_S, Q5_K_M, Q6_K, Q8_0, F16, F32

# VRAM helper flags
OFFLOAD_TEXT_ENCODER = True
DOWNLOAD_I2V_LIGHTNING_LORA = False

# ---- File-size tables (GB), checked from repo headers ----
WAN_I2V_QUANTS_GB = {
    'Q2_K': 5.30,
    'Q3_K_S': 6.52,
    'Q3_K_M': 7.18,
    'Q3_K_L': 7.79,
    'Q4_K_S': 8.75,
    'Q4_K_M': 9.65,
    'Q5_K_S': 10.14,
    'Q5_K_M': 10.79,
    'Q6_K': 12.00,
    'Q8_0': 15.41,
}

WAN_FLF2V_QUANTS_GB = {
    'Q2_K': 6.32,
    'Q3_K_S': 7.53,
    'Q3_K_M': 8.19,
    'Q4_0': 9.57,
    'Q4_1': 10.28,
    'Q4_K_S': 9.76,
    'Q4_K_M': 10.67,
    'Q5_0': 11.33,
    'Q5_1': 12.03,
    'Q5_K_S': 11.15,
    'Q5_K_M': 11.81,
    'Q6_K': 13.02,
    'Q8_0': 16.42,
}

UMT5_GGUF_GB = {
    'Q3_K_S': 2.86,
    'Q3_K_M': 3.06,
    'Q4_K_S': 3.50,
    'Q4_K_M': 3.66,
    'Q5_K_S': 4.05,
    'Q5_K_M': 4.15,
    'Q6_K': 4.67,
    'Q8_0': 6.04,
    'F16': 11.37,
    'F32': 22.73,
}

WAN22_VAE_GB = 1.41
WAN21_VAE_GB = 0.25
CLIP_VISION_H_GB = 1.26
I2V_LORA_EACH_GB = 1.23




def detect_total_vram_gb(default=AUTO_VRAM_FALLBACK_GB):
    try:
        import torch
        if torch.cuda.is_available():
            return torch.cuda.get_device_properties(0).total_memory / (1024**3)
    except Exception as e:
        print(f"VRAM detection failed ({e}); using fallback {default:.2f} GB")
    return default


def pick_best_quant(size_dict, budget_gb, overhead=1.10):
    items = sorted(size_dict.items(), key=lambda kv: kv[1])
    fitting = [(k, v) for k, v in items if (v * overhead) <= budget_gb]
    if fitting:
        return fitting[-1][0]
    return items[0][0]


def est_vram_gb(size_gb, overhead=1.10):
    return size_gb * overhead


def print_table(title, d):
    print('\n' + title)
    print('-' * len(title))
    for k, v in d.items():
        print(f"{k:8s}  file~{v:5.2f} GB   VRAM~{est_vram_gb(v):5.2f} GB")


# ---- Auto apply quant choice based on detected VRAM ----
TOTAL_VRAM_GB = detect_total_vram_gb()
AUTO_BUDGET_GB = TOTAL_VRAM_GB * AUTO_QUANT_VRAM_FRACTION
print(f"\nGPU VRAM detected: ~{TOTAL_VRAM_GB:.2f} GB | Auto budget (75%): ~{AUTO_BUDGET_GB:.2f} GB")

if AUTO_QUANT_BY_VRAM:
    i2v_candidates = list(WAN_I2V_QUANTS_GB.keys()) if DOWNLOAD_I2V else [I2V_HIGH_QUANT]
    flf_candidates = list(WAN_FLF2V_QUANTS_GB.keys()) if DOWNLOAD_FLF2V else [FLF2V_HIGH_QUANT]
    umt5_candidates = list(UMT5_GGUF_GB.keys()) if not OFFLOAD_TEXT_ENCODER else [UMT5_QUANT]

    best_fit = None
    best_any = None

    for i_q in i2v_candidates:
        for f_q in flf_candidates:
            for t_q in umt5_candidates:
                text_side = 0.0 if OFFLOAD_TEXT_ENCODER else est_vram_gb(UMT5_GGUF_GB[t_q])

                i2v_total = 0.0
                if DOWNLOAD_I2V:
                    i2v_total = 2 * est_vram_gb(WAN_I2V_QUANTS_GB[i_q]) + est_vram_gb(WAN22_VAE_GB) + text_side
                    if DOWNLOAD_I2V_LIGHTNING_LORA:
                        i2v_total += est_vram_gb(I2V_LORA_EACH_GB * 2)

                flf_total = 0.0
                if DOWNLOAD_FLF2V:
                    flf_total = (
                        2 * est_vram_gb(WAN_FLF2V_QUANTS_GB[f_q])
                        + est_vram_gb(WAN21_VAE_GB)
                        + est_vram_gb(CLIP_VISION_H_GB)
                        + text_side
                    )

                peak_total = max(i2v_total, flf_total)
                if not DOWNLOAD_I2V and not DOWNLOAD_FLF2V:
                    peak_total = text_side

                score = (
                    WAN_I2V_QUANTS_GB[i_q] if DOWNLOAD_I2V else 0.0,
                    WAN_FLF2V_QUANTS_GB[f_q] if DOWNLOAD_FLF2V else 0.0,
                    UMT5_GGUF_GB[t_q] if not OFFLOAD_TEXT_ENCODER else 0.0,
                )

                if (best_any is None) or (peak_total < best_any[0]):
                    best_any = (peak_total, i_q, f_q, t_q)

                if peak_total <= AUTO_BUDGET_GB:
                    if (best_fit is None) or (score > best_fit[0]) or (score == best_fit[0] and peak_total < best_fit[1]):
                        best_fit = (score, peak_total, i_q, f_q, t_q)

    chosen = best_fit if best_fit is not None else best_any
    I2V_HIGH_QUANT = I2V_LOW_QUANT = chosen[2]
    FLF2V_HIGH_QUANT = FLF2V_LOW_QUANT = chosen[3]
    if not OFFLOAD_TEXT_ENCODER:
        UMT5_QUANT = chosen[4]

    print(f"Auto quant active: I2V={I2V_HIGH_QUANT}, FLF2V={FLF2V_HIGH_QUANT}, UMT5={UMT5_QUANT}")

print_table('Wan2.2 I2V 14B GGUF quants (high/low)', WAN_I2V_QUANTS_GB)
print_table('Wan2.2 First/Last (Fun Camera) GGUF quants (high/low)', WAN_FLF2V_QUANTS_GB)
print_table('UMT5 encoder GGUF quants', UMT5_GGUF_GB)

print('\nSelected:')
print('  DOWNLOAD_I2V              =', DOWNLOAD_I2V)
print('  DOWNLOAD_FLF2V            =', DOWNLOAD_FLF2V)
print('  I2V_HIGH_QUANT / LOW      =', I2V_HIGH_QUANT, '/', I2V_LOW_QUANT)
print('  FLF2V_HIGH_QUANT / LOW    =', FLF2V_HIGH_QUANT, '/', FLF2V_LOW_QUANT)
print('  UMT5_QUANT                =', UMT5_QUANT)
print('  OFFLOAD_TEXT_ENCODER      =', OFFLOAD_TEXT_ENCODER)
print('  DOWNLOAD_I2V_LIGHTNING_LORA =', DOWNLOAD_I2V_LIGHTNING_LORA)

# Rough GPU totals per mode
text_side = est_vram_gb(UMT5_GGUF_GB[UMT5_QUANT])
if OFFLOAD_TEXT_ENCODER:
    text_side = 0.0

i2v_total = 0.0
if DOWNLOAD_I2V:
    i2v_total += est_vram_gb(WAN_I2V_QUANTS_GB[I2V_HIGH_QUANT])
    i2v_total += est_vram_gb(WAN_I2V_QUANTS_GB[I2V_LOW_QUANT])
    i2v_total += est_vram_gb(WAN22_VAE_GB)
    i2v_total += text_side
    if DOWNLOAD_I2V_LIGHTNING_LORA:
        i2v_total += est_vram_gb(I2V_LORA_EACH_GB * 2)

flf_total = 0.0
if DOWNLOAD_FLF2V:
    flf_total += est_vram_gb(WAN_FLF2V_QUANTS_GB[FLF2V_HIGH_QUANT])
    flf_total += est_vram_gb(WAN_FLF2V_QUANTS_GB[FLF2V_LOW_QUANT])
    flf_total += est_vram_gb(WAN21_VAE_GB)
    flf_total += est_vram_gb(CLIP_VISION_H_GB)
    flf_total += text_side

print(f"\nRough GPU VRAM (I2V mode):   ~{i2v_total:.2f} GB")
print(f"Rough GPU VRAM (FLF2V mode): ~{flf_total:.2f} GB")
print('Note: these are rough weight-only estimates; frame count/resolution can increase peak usage a lot.')


## 3) Model Download
Download the required model files (and optional components) into ComfyUI folders.


In [ ]:
# @title 3) Download models (Wan2.2 I2V + First/Last in one notebook)
import os

COMFY = '/content/ComfyUI'

UNET_DIR = f'{COMFY}/models/unet'
DIFF_DIR = f'{COMFY}/models/diffusion_models'
TE_DIR   = f'{COMFY}/models/text_encoders'
CLIP_DIR = f'{COMFY}/models/clip'
VAE_DIR  = f'{COMFY}/models/vae'
CLIPV_DIR = f'{COMFY}/models/clip_vision'
LORA_DIR = f'{COMFY}/models/loras'
WORKFLOW_DIR = f'{COMFY}/user/default/workflows'

for d in [UNET_DIR, DIFF_DIR, TE_DIR, CLIP_DIR, VAE_DIR, CLIPV_DIR, LORA_DIR, WORKFLOW_DIR]:
    os.makedirs(d, exist_ok=True)


def dl(url, outdir, fname):
    outpath = os.path.join(outdir, fname)
    if os.path.exists(outpath):
        print('Already exists:', outpath)
        return
    print('Downloading:', fname)

    hf_token = os.environ.get('HUGGINGFACE_TOKEN') or os.environ.get('HF_TOKEN')
    if hf_token and 'huggingface.co' in url:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --header="Authorization: Bearer {hf_token}" "{url}" -d "{outdir}" -o "{fname}"
    else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{outdir}" -o "{fname}"


def link_if_missing(src, dst):
    if not os.path.exists(dst):
        !ln -s "{src}" "{dst}"


# Common text encoder (GGUF)
umt5_fname = f"umt5-xxl-encoder-{UMT5_QUANT}.gguf"
umt5_url = f"https://huggingface.co/city96/umt5-xxl-encoder-gguf/resolve/main/{umt5_fname}"
dl(umt5_url, TE_DIR, umt5_fname)
link_if_missing(f"{TE_DIR}/{umt5_fname}", f"{CLIP_DIR}/{umt5_fname}")

# Official VAE files
wan22_vae_fname = 'wan2.2_vae.safetensors'
wan22_vae_url = 'https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan2.2_vae.safetensors'
dl(wan22_vae_url, VAE_DIR, wan22_vae_fname)

wan21_vae_fname = 'wan_2.1_vae.safetensors'
wan21_vae_url = 'https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors'
dl(wan21_vae_url, VAE_DIR, wan21_vae_fname)

clipv_fname = 'clip_vision_h.safetensors'
clipv_url = 'https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/clip_vision/clip_vision_h.safetensors'
dl(clipv_url, CLIPV_DIR, clipv_fname)

# I2V 14B GGUF (high+low)
i2v_high_fname = f"wan2.2_i2v_high_noise_14B_{I2V_HIGH_QUANT}.gguf"
i2v_low_fname  = f"wan2.2_i2v_low_noise_14B_{I2V_LOW_QUANT}.gguf"

if DOWNLOAD_I2V:
    i2v_high_url = f"https://huggingface.co/bullerwins/Wan2.2-I2V-A14B-GGUF/resolve/main/{i2v_high_fname}"
    i2v_low_url  = f"https://huggingface.co/bullerwins/Wan2.2-I2V-A14B-GGUF/resolve/main/{i2v_low_fname}"
    dl(i2v_high_url, UNET_DIR, i2v_high_fname)
    dl(i2v_low_url, UNET_DIR, i2v_low_fname)
    link_if_missing(f"{UNET_DIR}/{i2v_high_fname}", f"{DIFF_DIR}/{i2v_high_fname}")
    link_if_missing(f"{UNET_DIR}/{i2v_low_fname}", f"{DIFF_DIR}/{i2v_low_fname}")

# First/Last (Fun Camera A14B GGUF, high+low)
flf_high_fname = f"Wan2.2-Fun-A14B-Control-Camera-HighNoise-{FLF2V_HIGH_QUANT}.gguf"
flf_low_fname  = f"Wan2.2-Fun-A14B-Control-Camera-LowNoise-{FLF2V_LOW_QUANT}.gguf"

if DOWNLOAD_FLF2V:
    flf_high_url = f"https://huggingface.co/QuantStack/Wan2.2-Fun-A14B-Control-Camera-GGUF/resolve/main/HighNoise/{flf_high_fname}"
    flf_low_url  = f"https://huggingface.co/QuantStack/Wan2.2-Fun-A14B-Control-Camera-GGUF/resolve/main/LowNoise/{flf_low_fname}"
    dl(flf_high_url, UNET_DIR, flf_high_fname)
    dl(flf_low_url, UNET_DIR, flf_low_fname)
    link_if_missing(f"{UNET_DIR}/{flf_high_fname}", f"{DIFF_DIR}/{flf_high_fname}")
    link_if_missing(f"{UNET_DIR}/{flf_low_fname}", f"{DIFF_DIR}/{flf_low_fname}")

# Optional I2V 4-step LoRAs (official Comfy-Org repackaged)
if DOWNLOAD_I2V and DOWNLOAD_I2V_LIGHTNING_LORA:
    lora_high = 'wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors'
    lora_low  = 'wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors'
    lora_high_url = f"https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/{lora_high}"
    lora_low_url  = f"https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/{lora_low}"
    dl(lora_high_url, LORA_DIR, lora_high)
    dl(lora_low_url, LORA_DIR, lora_low)

# Official workflows
if DOWNLOAD_I2V:
    dl('https://raw.githubusercontent.com/comfyanonymous/ComfyUI_examples/master/wan22/image_to_video_wan22_14B.json', WORKFLOW_DIR, 'image_to_video_wan22_14B.json')
if DOWNLOAD_FLF2V:
    dl('https://raw.githubusercontent.com/comfyanonymous/ComfyUI_examples/master/wan/camera_image_to_video_wan_example.json', WORKFLOW_DIR, 'camera_image_to_video_wan_example.json')

print('Done downloading models.')
print('UMT5 GGUF:', umt5_fname)
print('VAE files:', wan22_vae_fname, 'and', wan21_vae_fname)
print('ClipVision:', clipv_fname)
if DOWNLOAD_I2V:
    print('I2V models:', i2v_high_fname, '|', i2v_low_fname)
if DOWNLOAD_FLF2V:
    print('First/Last models:', flf_high_fname, '|', flf_low_fname)


# ---- Lenovo UltraReal LoRA pack (Civitai) ----
DOWNLOAD_LENOVO_ULTRAREAL_LORAS = True
CIVITAI_API_TOKEN = os.environ.get('CIVITAI_API_TOKEN', '').strip()  # optional, helps if Civitai returns 403.
CURRENT_BASE_MODELS = ['Wan Video 14B t2v']

if DOWNLOAD_LENOVO_ULTRAREAL_LORAS:
    import json
    import subprocess
    import urllib.request
    import urllib.parse

    LORA_DIR = f'{COMFY}/models/loras'
    os.makedirs(LORA_DIR, exist_ok=True)

    def add_civitai_token(url, token):
        if not token:
            return url
        parts = urllib.parse.urlsplit(url)
        query = urllib.parse.parse_qsl(parts.query, keep_blank_values=True)
        if not any(k == 'token' for k, _ in query):
            query.append(('token', token))
        return urllib.parse.urlunsplit((parts.scheme, parts.netloc, parts.path, urllib.parse.urlencode(query), parts.fragment))

    def dl_civitai(url, outdir, fname, token=''):
        outpath = os.path.join(outdir, fname)
        if os.path.exists(outpath):
            print('Already exists:', outpath)
            return True

        final_url = add_civitai_token(url, token)
        print('Downloading LoRA:', fname)
        cmd = ['aria2c', '--console-log-level=error', '-c', '-x', '8', '-s', '8', '-k', '1M', final_url, '-d', outdir, '-o', fname]
        rc = subprocess.run(cmd, check=False).returncode
        if rc != 0 and not token:
            print('  Download failed without token. Set CIVITAI_API_TOKEN env var and retry Cell 3.')
        return rc == 0

    try:
        req = urllib.request.Request(
            'https://civitai.com/api/v1/models/1662740',
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        with urllib.request.urlopen(req, timeout=30) as resp:
            civitai_model = json.loads(resp.read().decode('utf-8'))

        versions = civitai_model.get('modelVersions', [])
        print('\nLenovo UltraReal LoRA versions found:', len(versions))
        if not CURRENT_BASE_MODELS:
            print('Current notebook base-model tags: (none matched explicitly)')
        else:
            print('Current notebook base-model tags:', ', '.join(CURRENT_BASE_MODELS))

        for mv in versions:
            base = mv.get('baseModel', 'Unknown')
            match = base in CURRENT_BASE_MODELS
            mark = 'MATCH' if match else 'OTHER'
            files = [f for f in mv.get('files', []) if f.get('type') == 'Model']
            print(f"- [{mark}] {base} :: {mv.get('name', 'Unnamed version')} ({len(files)} file(s))")

        ok_count = 0
        total_count = 0
        matched_versions = [mv for mv in versions if mv.get('baseModel', 'Unknown') in CURRENT_BASE_MODELS]

        if not matched_versions:
            print('No MATCH versions found for this notebook base-model tag set. Skipping LoRA download.')

        for mv in matched_versions:
            for fobj in mv.get('files', []):
                if fobj.get('type') != 'Model':
                    continue
                total_count += 1
                raw_name = (fobj.get('name') or '').strip()
                base_tag = ''.join(ch if ch.isalnum() else '_' for ch in str(mv.get('baseModel', 'unknown'))).strip('_').lower() or 'unknown'
                fname = f"lenovo_ultrareal_{base_tag}_v{mv.get('id', 'x')}_f{fobj.get('id', 'x')}.safetensors"
                if raw_name and raw_name != fname:
                    print(f"  Civitai file name: {raw_name} -> saved as {fname}")
                if dl_civitai(fobj.get('downloadUrl', ''), LORA_DIR, fname, CIVITAI_API_TOKEN):
                    ok_count += 1

        print(f"\nLenovo UltraReal LoRA download summary (MATCH only): {ok_count}/{total_count} files ready in {LORA_DIR}")

    except Exception as e:
        print('Failed to fetch Lenovo UltraReal LoRA metadata from Civitai:', e)
        print('Tip: this may require CIVITAI_API_TOKEN or temporary Civitai availability.')


## 4) Download Check
Verify downloaded file sizes and calculate a rough VRAM estimate from real files.


In [ ]:
# @title 4) Verify downloads: actual file sizes and VRAM estimate
import os


def size_gb(path):
    return os.path.getsize(path) / (1024**3)


def est_vram_from_file(path, overhead=1.10):
    return size_gb(path) * overhead


paths = [
    ('/content/ComfyUI/models/text_encoders', f"umt5-xxl-encoder-{UMT5_QUANT}.gguf"),
    ('/content/ComfyUI/models/vae', 'wan2.2_vae.safetensors'),
    ('/content/ComfyUI/models/vae', 'wan_2.1_vae.safetensors'),
    ('/content/ComfyUI/models/clip_vision', 'clip_vision_h.safetensors'),
]

if DOWNLOAD_I2V:
    paths.append(('/content/ComfyUI/models/unet', f"wan2.2_i2v_high_noise_14B_{I2V_HIGH_QUANT}.gguf"))
    paths.append(('/content/ComfyUI/models/unet', f"wan2.2_i2v_low_noise_14B_{I2V_LOW_QUANT}.gguf"))

if DOWNLOAD_FLF2V:
    paths.append(('/content/ComfyUI/models/unet', f"Wan2.2-Fun-A14B-Control-Camera-HighNoise-{FLF2V_HIGH_QUANT}.gguf"))
    paths.append(('/content/ComfyUI/models/unet', f"Wan2.2-Fun-A14B-Control-Camera-LowNoise-{FLF2V_LOW_QUANT}.gguf"))

if DOWNLOAD_I2V and DOWNLOAD_I2V_LIGHTNING_LORA:
    paths.append(('/content/ComfyUI/models/loras', 'wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors'))
    paths.append(('/content/ComfyUI/models/loras', 'wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors'))

print('Files:')
total_vram = 0.0
for d, f in paths:
    p = os.path.join(d, f)
    if not os.path.exists(p):
        print('MISSING:', p)
        continue
    s = size_gb(p)
    v = est_vram_from_file(p)
    total_vram += v
    print(f"- {p}\n  size~{s:.2f} GB  -> VRAM(weights)~{v:.2f} GB")

print(f"\nSum VRAM(weights) if all kept on GPU at once: ~{total_vram:.2f} GB")
print('Reminder: in real workflows, active VRAM depends on mode, frame count, resolution and offloading.')


## 5) Launch ComfyUI
Start ComfyUI and use the Cloudflare Tunnel URL shown in the output.


In [ ]:
# @title 5) Launch ComfyUI (Cloudflare Tunnel) - stable mode
import subprocess, threading, time, socket, os, re, sys, urllib.request

%cd /content/ComfyUI

# Install cloudflared (if not installed yet)
if not os.path.exists("cloudflared-linux-amd64.deb"):
    !wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb


def wait_port(host="127.0.0.1", port=8188, timeout=240):
    t0 = time.time()
    while time.time() - t0 < timeout:
        try:
            with socket.create_connection((host, port), timeout=1):
                return True
        except OSError:
            time.sleep(0.5)
    return False


def verify_tunnel_url(url, timeout=10):
    base = url.rstrip('/')
    candidates = [
        base + '/',
        base + '/api/system_stats',
        base + '/object_info',
    ]
    for c in candidates:
        try:
            req = urllib.request.Request(c, headers={'User-Agent': 'Mozilla/5.0'}, method='GET')
            with urllib.request.urlopen(req, timeout=timeout) as r:
                code = r.getcode()
                if 200 <= code < 500:
                    return True
        except Exception:
            pass
    return False


def wait_reachable_stable(url, proc, warmup_timeout=180, stable_successes=3, probe_interval=2.5):
    t0 = time.time()
    ok_streak = 0
    while time.time() - t0 < warmup_timeout:
        if proc.poll() is not None:
            return False
        if verify_tunnel_url(url, timeout=10):
            ok_streak += 1
            if ok_streak >= stable_successes:
                return True
        else:
            ok_streak = 0
        time.sleep(probe_interval)
    return False


def start_cloudflare_tunnel_once(port=8188, protocol='http2', read_timeout=150, warmup_timeout=180):
    print(f"\nStarting Cloudflare Quick Tunnel (protocol={protocol})...\n")
    sys.stdout.flush()

    cmd = [
        'cloudflared', 'tunnel',
        '--no-autoupdate',
        '--url', f'http://127.0.0.1:{port}',
        '--protocol', protocol,
        '--loglevel', 'info',
    ]

    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
    )

    tunnel_patterns = [
        re.compile(r'https://[a-z0-9-]+\.trycloudflare\.com', re.I),
        re.compile(r'https://[a-z0-9-]+\.cfargotunnel\.com', re.I),
    ]
    ignore_patterns = [
        re.compile(r'https://www\.cloudflare\.com/website-terms/?', re.I),
    ]

    t0 = time.time()
    url = None

    while time.time() - t0 < read_timeout:
        line = proc.stdout.readline()
        if not line:
            if proc.poll() is not None:
                break
            time.sleep(0.1)
            continue

        s = line.strip()

        if any(ip.search(s) for ip in ignore_patterns):
            continue

        for pat in tunnel_patterns:
            m = pat.search(s)
            if m:
                url = m.group(0)
                break
        if url:
            break

    if not url:
        print('Failed to parse tunnel URL.')
        if proc.poll() is None:
            proc.terminate()
            try:
                proc.wait(timeout=5)
            except Exception:
                proc.kill()
        return None, None

    print(f'Found tunnel URL: {url}')
    print('Waiting for Cloudflare propagation and stable readiness...')

    if wait_reachable_stable(url, proc, warmup_timeout=warmup_timeout, stable_successes=3, probe_interval=2.5):
        print('\n--------------------------------------------------')
        print('YOUR LINK:', url)
        print('--------------------------------------------------\n')
        return proc, url

    print('Tunnel URL did not become stably reachable in time.')
    if proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except Exception:
            proc.kill()
    return None, url


def start_tunnel_with_retries(port=8188, max_attempts=6):
    protocols = ['http2', 'quic']
    for attempt in range(1, max_attempts + 1):
        proto = protocols[(attempt - 1) % len(protocols)]
        print(f"\n== Tunnel attempt {attempt}/{max_attempts} ==")
        proc, url = start_cloudflare_tunnel_once(
            port=port,
            protocol=proto,
            read_timeout=150,
            warmup_timeout=180,
        )
        if proc is not None and url:
            return proc, url
        time.sleep(min(8, 2 + attempt))
    return None, None


def stop_proc_safely(proc):
    if proc is None:
        return
    if proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except Exception:
            proc.kill()


def tunnel_thread(port=8188):
    if not wait_port('127.0.0.1', port, timeout=240):
        print('Timed out waiting for ComfyUI port', port)
        return

    print('\nComfyUI port is open. Creating stable tunnel...\n')

    while True:
        proc, url = start_tunnel_with_retries(port=port, max_attempts=6)
        if proc is None:
            print('Failed to establish a reachable Cloudflare tunnel automatically.')
            print('Rerun this cell to retry with a fresh tunnel session.')
            return

        unhealthy_streak = 0
        while proc.poll() is None:
            time.sleep(15)
            if verify_tunnel_url(url, timeout=8):
                unhealthy_streak = 0
            else:
                unhealthy_streak += 1
                print(f'Cloudflare tunnel health check failed ({unhealthy_streak}/3).')
                if unhealthy_streak >= 3:
                    print('Tunnel became unhealthy. Recreating...')
                    stop_proc_safely(proc)
                    break

        if proc.poll() is not None:
            rc = proc.returncode
            print(f'Cloudflare tunnel exited (code={rc}). Recreating...')


threading.Thread(target=tunnel_thread, daemon=True, args=(8188,)).start()

print('Starting ComfyUI... link will appear above.')
!python main.py --dont-print-server --port 8188 --lowvram --preview-method auto
